In [ ]:
!pip install textdistance
!pip install py2neo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 4.0 MB/s 


# 1 Token-level distance function

We use algorithms in `textdistance` to compute the distance between two strings in token-level. An example is `jaccard` algorithm:

In [ ]:
import textdistance as td 

In [ ]:
td.jaccard('感冒','感,;sdf冒')

0.2857142857142857

## One Mention, Many Candidates
Since in real practice we should compute a mention word against many entities from a knowledge base, we further implement this interface.

In [ ]:
import numpy as np
def one2many(dist_func):
    def new_func(mention:str, candidates: list):
        f = lambda candidate: dist_func(mention,candidate)
        return np.array(list(map(f, candidates)))

    return new_func

In [ ]:
jaccard_many = one2many(td.jaccard)

In [ ]:
jaccard_many('感冒',['感冒','流行感冒','小儿感冒'])

array([1. , 0.5, 0.5])

# 2 Connect to Neo4j KG
Connect to Neo4j KG and load the disease entities.

In [ ]:
from py2neo import Graph
password = None # set your password here
graph = Graph("neo4j+s://f54cadff.databases.neo4j.io:7687", auth=("neo4j", password))

In [ ]:
def query_graph(entity_type:str):
    '''
    return: query results in List
    '''
    cql = f'match (n:`{entity_type}`) return n.name'
    ret = list(graph.run(cql))
    f = lambda record: record.values()[0]
    ret = list(map(f,ret))
    return ret

In [ ]:
result = query_graph('疾病')

In [ ]:
len(result)

8807

In [ ]:
result[0]

'放射性视网膜病变'

# 3 Compute similarity between mention and candidates

In [ ]:
similarities = jaccard_many('感冒',result)

In [ ]:
id = np.argmax(similarities)
id

5686

In [ ]:
result[id]

'感冒'

In [ ]:
def topK_candidates(k:int, mention, candidates):
    similarities = jaccard_many(mention,candidates)
    ids = np.argpartition(similarities, -k)[-k:]

    result = [candidates[a] for a in ids]
    f = lambda x : td.jaccard(mention,x)
    return sorted(result, key=f, reverse= True)
    

In [ ]:
topK_candidates(3,'感冒',result)

['感冒', '风寒感冒', '小儿感冒']

In [ ]:
!pip install git+https://github.com/leoxiang66/text-similarity.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/leoxiang66/text-similarity.git to /tmp/pip-req-build-p_kjfvhm
  Running command git clone -q https://github.com/leoxiang66/text-similarity.git /tmp/pip-req-build-p_kjfvhm


In [ ]:
from textsimilarity import textsimilarity as ts

In [ ]:
ts.top_K_similarity_between_one_mention_and_many_candidates(10,'感冒',result)

['感冒',
 '风热感冒',
 '小儿感冒',
 '风寒感冒',
 '肠胃型感冒',
 '病毒性感冒',
 '流行性感冒',
 '郁冒',
 '副流行性感冒',
 '小儿流行性感冒']